In [20]:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/14JY1DjWv6MaPmoSmyihDWRcVRkiTqmJO?usp=sharing)

SyntaxError: ignored

In [ ]:
!nvidia-smi

Sat May  7 15:29:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.3 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 880 kB 38.5 MB/s 
     |████████████████████████████████| 596 kB 41.8 MB/s 
     |████████████████████████████████| 6.6 MB 40.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=18505935560623593706611607ed2459a738c38e838054b2331d633ce3a25793
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Finetuning

In [ ]:
pip install simpletransformers datasets tqdm pandas

In [ ]:
import pandas as pd
dataset_df = pd.read_csv("ArgKP_dataset.csv")

In [ ]:
dataset_df.head

<bound method NDFrame.head of                                                topic  \
0      Assisted suicide should be a criminal offence   
1      Assisted suicide should be a criminal offence   
2      Assisted suicide should be a criminal offence   
3      Assisted suicide should be a criminal offence   
4      Assisted suicide should be a criminal offence   
...                                              ...   
24088       We should subsidize vocational education   
24089       We should subsidize vocational education   
24090       We should subsidize vocational education   
24091       We should subsidize vocational education   
24092       We should subsidize vocational education   

                                                argument  \
0      `people reach their limit when it comes to the...   
1      a cure or treatment may be discovered shortly ...   
2      a cure or treatment may be discovered shortly ...   
3      a cure or treatment may be discovered shortly ... 

In [ ]:
dataset_df.columns

Index(['topic', 'argument', 'key_point', 'stance', 'label'], dtype='object')

In [ ]:
dataset_df_new = dataset_df.filter(['argument', 'key_point'], axis=1)

In [ ]:
dataset_df_new.head(5)

,argument,key_point
0,`people reach their limit when it comes to the...,Assisted suicide gives dignity to the person t...
1,a cure or treatment may be discovered shortly ...,Assisted suicide allows people to solicit some...
2,a cure or treatment may be discovered shortly ...,Assisted suicide is akin to killing someone
3,a cure or treatment may be discovered shortly ...,Assisted suicide should not be allowed because...
4,a cure or treatment may be discovered shortly ...,Assisted suicide violates the doctor's role


In [ ]:
for i in dataset_df_new.index:
  arg = dataset_df_new['argument'][i]
  if arg[-1] != '.':
    modified_arg = arg + '. This means '
    dataset_df_new.at[i, 'argument'] = modified_arg
  else:
    modified_arg = arg + ' This means '
    dataset_df_new.at[i, 'argument'] = modified_arg

In [ ]:
dataset_df_new.at[110, 'argument']

'Assisted suicide is a criminal offence as it centers on the intentional death of a human being. This means '

In [ ]:
# Renaming the columns
dataset_df_new.columns = ["input_text","target_text"]

In [ ]:
# Adding a prefix. Here we shall keep "paraphrase" as a prefix.
dataset_df_new["prefix"] = "paraphrase"

In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data = train_test_split(dataset_df_new,test_size=0.1)

In [ ]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    "adam_epsilon": 1e-08,
    "eval_batch_size": 6,
    "fp_16": False,
    "gradient_accumulation_steps": 16,
    "learning_rate": 0.0003,
    "max_grad_norm": 1.0,
    "n_gpu": 1,
    "seed": 42,
    "train_batch_size": 6,
    "warmup_steps": 0,
    "weight_decay": 0.0,
}

In [ ]:
from simpletransformers.t5 import T5Model
import sklearn
model = T5Model("t5","t5-small", args=args)

In [ ]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/21683 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/3614 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 4:   0%|          | 0/3614 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/3614 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/3614 [00:00<?, ?it/s]

(900, 0.8105092829926354)

In [ ]:
from simpletransformers.t5 import T5Model
import os
root_dir = os.getcwd()
trained_model_path = os.path.join(root_dir,"outputs")
args = {
"overwrite_output_dir": True,
"max_seq_length": 256,
"max_length": 50,
"top_k": 50,
"top_p": 0.95,
"num_return_sequences": 5
}
trained_model = T5Model("t5",trained_model_path,args=args)

In [ ]:
prefix = "paraphrase"
pred = trained_model.predict([f"{prefix}: we shouldnt legalize cannabis like any drug its addictive and can be abused and affect the senses"])
print(pred)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3516: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/5 [00:00<?, ?it/s]

[['Cannabis is a gateway-drug/addictive', 'Cannabis harms the body', 'Cannabis is a gateway-drug/addictive', 'Cannabis harms the body', 'Cannabis harms the body']]


In [ ]:
print(test_data[0:1]['input_text'])

18304    we shouldnt legalize cannabis like any drug it...
Name: input_text, dtype: object


In [ ]:
from google.colab import files
files.download("/content/t5-model.zip")

In [ ]:
!zip -r /content/t5-model.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-675-epoch-3/ (stored 0%)
  adding: content/outputs/checkpoint-675-epoch-3/special_tokens_map.json (deflated 83%)
  adding: content/outputs/checkpoint-675-epoch-3/spiece.model (deflated 48%)
  adding: content/outputs/checkpoint-675-epoch-3/config.json (deflated 63%)
  adding: content/outputs/checkpoint-675-epoch-3/model_args.json (deflated 63%)
  adding: content/outputs/checkpoint-675-epoch-3/pytorch_model.bin (deflated 7%)
  adding: content/outputs/checkpoint-675-epoch-3/scheduler.pt (deflated 51%)
  adding: content/outputs/checkpoint-675-epoch-3/tokenizer_config.json (deflated 79%)
  adding: content/outputs/checkpoint-675-epoch-3/optimizer.pt (deflated 15%)
  adding: content/outputs/checkpoint-675-epoch-3/training_args.bin (deflated 49%)
  adding: content/outputs/special_tokens_map.json (deflated 83%)
  adding: content/outputs/spiece.model (deflated 48%)
  adding: content/outputs/checkpoint-450-epoch-2/ (store

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive
